# Task 1 Building CNN-based model using Keras and Tensorflow and Fine-tuned Resnet with Pytorch that is capable of classifying anatomical structure in 2D fetal ultrasound images on AWS Sagemaker 


A zip file named "Task1.zip" is added to JuypterLab notebook in AWS Sagemaker. Form extraction, installing unzip

In [ ]:
!conda install -y -c conda-forge unzip

In [ ]:
# Unzipping Task1.zip
!unzip Task1

In [ ]:
# Our training images are in Classification/images. Now, getting all image files for this dir
import os
current_dir = "Classification/images"
image_data = []
for i in os.listdir(current_dir):
    image_data.append(current_dir + "/" + i)
len(image_data)

In [ ]:
# reading csv file
import pandas as pd
train_data = pd.read_csv("image_label.csv")

In [ ]:
train_data.head()

# Updating the train_data dataframe Image_name column with dir locations

In [ ]:
for i in range(len(train_data.Image_name)):
    train_data.Image_name.iloc[i] = "Classification/images/" + str(train_data.Image_name.iloc[i]) + ".png"

In [ ]:
train_data.shape

In [ ]:
train_data.head()

In [ ]:
train_data.Image_name.iloc[0]

In [ ]:
# Our testing images without label are in Classification/External Test images. Now, getting all image files for this dir
import os
current_dir = "Classification/External Test images"
test_data = []
for i in os.listdir(current_dir):
    test_data.append(current_dir + "/" + i)
len(test_data)

In [ ]:
from PIL import Image
idx_train = []
for i in range(len(train_data.Image_name)):
    with Image.open(train_data.Image_name.iloc[i]) as img:
      if img.verify():
          print("Not an image encountered")
          idx_train.append(i)
idx_train

In [ ]:
from PIL import Image
idx_test = []
for i in range(len(test_data)):
    with Image.open(test_data[i]) as img:
      if img.verify():
          print("Not an image encountered")
          idx_test.append(i)
idx_test

In [ ]:
# Understanding the dataframe
train_data.describe()

In [ ]:
# Checking for any missing labels
train_data['Plane'].isnull().sum()

In [ ]:
# Determining the types of unique classes
classes = train_data.Plane.unique()
classes

In [ ]:
classes_dict = {val: train_data[train_data['Plane'] == val]['Image_name'].tolist() for val in classes}
f_brain = classes_dict["Fetal brain"]
f_femur = classes_dict["Fetal femur"]
f_thorax = classes_dict["Fetal thorax"]
f_abdomen = classes_dict["Fetal abdomen"]

In [ ]:
# Making sure we have identified all the samples
len(f_brain) + len(f_femur) + len(f_thorax) + len(f_abdomen)

# Understanding the distribution of images in terms of their size

In [ ]:
import cv2
import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

In [ ]:
widths = []
heights = []

for i in range(len(train_data.Image_name)):
    img = cv2.imread(train_data.Image_name.iloc[i], cv2.IMREAD_GRAYSCALE)
    h, w = img.shape
    widths.append(w)
    heights.append(h)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].hist(widths, bins=20, color='blue', edgecolor='black')
axes[0].set_title('Image Width Distribution')
axes[0].set_xlabel('Width')
axes[0].set_ylabel('Frequency')
axes[1].hist(heights, bins=20, color='green', edgecolor='black')
axes[1].set_title('Image Height Distribution')
axes[1].set_xlabel('Height')
axes[1].set_ylabel('Frequency')
plt.tight_layout()
plt.show()


# Now checking the unique classes distribution sizes and if there is any imbalance in the data

In [ ]:
num_f_brain = len(f_brain)
num_f_femur = len(f_femur)
num_f_thorax = len(f_thorax)
num_f_abdomen = len(f_abdomen)
total_images = len(f_brain) + len(f_femur) + len(f_thorax) + len(f_abdomen)
print("brain:", num_f_brain)
print("femur:", num_f_femur)
print("thorax:", num_f_thorax)
print("abdomen:", num_f_abdomen)

In [ ]:
classes = train_data.Plane.unique()
counts = [num_f_brain, num_f_femur, num_f_thorax, num_f_abdomen]
plt.bar(classes, counts)
plt.xlabel('Class')
plt.ylabel('Number of Images')
plt.title('Distribution of Images in Each Class')
plt.show()

In [ ]:
def mask_analysis():
    image_files = train_data.Image_name
    images = [cv2.imread(img_f, cv2.IMREAD_GRAYSCALE) for img_f in image_files]
    masks = []
    for img in images:
        mask = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
        masks.append(mask)
        mask[mask != 255] = 0
    for i in range(3):
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 3, 1)
        plt.imshow(images[i], cmap='gray')
        plt.title('Original Image')

        plt.subplot(1, 3, 2)
        plt.imshow(masks[i], cmap='gray')
        plt.title('Generated Mask')

        plt.show()

# Display masks of normal images
mask_analysis()

# Now making all classes sample size equal to handle the imbalance data

# Technique used - Oversampling

In [ ]:
import random

In [ ]:
target_num_samples = max(num_f_brain, num_f_femur, num_f_thorax, num_f_abdomen)
oversampled_brain_image_paths = random.choices(f_brain, k = target_num_samples)
oversampled_femur_image_paths = random.choices(f_femur, k = target_num_samples)
oversampled_thorax_image_paths = random.choices(f_thorax, k = target_num_samples)
oversampled_abdomen_image_paths = random.choices(f_abdomen, k = target_num_samples)
num_f_brain = len(oversampled_brain_image_paths)
num_f_femur = len(oversampled_femur_image_paths)
num_f_thorax = len(oversampled_thorax_image_paths)
num_f_abdomen = len(oversampled_abdomen_image_paths)

print("Number of images for each class after Oversampling")
print("Brain:", num_f_brain)
print("Femur:", num_f_femur)
print("Thorax:", num_f_thorax)
print("Abdomen:", num_f_abdomen)

In [ ]:
counts = [num_f_brain, num_f_femur, num_f_thorax, num_f_abdomen]
plt.bar(classes, counts)
plt.xlabel('Class')
plt.ylabel('Number of Images')
plt.title('Distribution of Images in Each Class after Oversampling')
plt.show()

# Now resizing images and normalizing to [0, 1]

In [ ]:
img_size = (224, 224)
import numpy as np
from PIL import Image
def resize_images(image_paths):
    images = []
    for filepath in image_paths:
        img = Image.open(filepath)
        img = np.array(img.resize(img_size))
        img_array = np.array(img) / 255.0
        images.append(img)
    return np.array(images)

processed_brain_images = resize_images(oversampled_brain_image_paths)
processed_femur_images = resize_images(oversampled_femur_image_paths)
processed_thorax_images = resize_images(oversampled_thorax_image_paths)
processed_abdomen_images = resize_images(oversampled_abdomen_image_paths)

# Now performing the Data Augmentation using ImageDataGenerator to improve generalization of data

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [ ]:
# ImageDataGenerator with augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
#function to perform augmentation
def augment_images(images_array, num_augmented_per_image=10):
    reshaped_images = images_array.reshape(images_array.shape[0], 224, 224, 1)
    datagen.fit(reshaped_images)
    augmented_images = []
    for x_batch in datagen.flow(reshaped_images, batch_size=1):
        augmented_images.extend(x_batch)
        if len(augmented_images) >= len(reshaped_images) * num_augmented_per_image:
            break
    return np.array(augmented_images)

In [ ]:
# calling augmentation_image with augmentation_factor = 5
augmented_brain_images = augment_images(processed_brain_images, num_augmented_per_image = 5)
augmented_femur_images = augment_images(processed_femur_images, num_augmented_per_image = 5)
augmented_thorax_images = augment_images(processed_thorax_images, num_augmented_per_image = 5)
augmented_abdomen_images = augment_images(processed_abdomen_images, num_augmented_per_image = 5)

# Now creating labels for classes as 0, 1, 2, 3
# We can use label encoder for it. Since the number of classes are limited to 4, creating labels manually

In [ ]:
# label 0 , 1, 2, 3 -> brain, femur, thorax, abdomen
brain_labels = np.zeros((len(augmented_brain_images),), dtype=int)
femur_labels = np.ones((len(augmented_femur_images),), dtype=int)
thorax_labels = np.full((len(augmented_thorax_images),), 2)
abdomen_labels = np.full((len(augmented_abdomen_images),), 3)

In [ ]:
# Concatenate augmented images into a single
X = np.concatenate([augmented_brain_images, augmented_femur_images, augmented_thorax_images, augmented_abdomen_images])

In [ ]:
if len(X.shape) == 3:
    X = np.expand_dims(X, axis=-1)

In [ ]:
# Concatenate labels into a single
y = np.concatenate([brain_labels, femur_labels, thorax_labels, abdomen_labels])

In [ ]:
X.shape, y.shape

# Now preparing data for training by splitting it

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
# Further split the temporary data into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.2, random_state = 42)

In [ ]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape, X_test.shape, y_test.shape

# Builing and Training the model using CNN

In [ ]:
from tensorflow.keras import regularizers
from tensorflow.keras.layers import (BatchNormalization, Conv2D, Dense, Dropout, Flatten, GlobalAveragePooling2D, MaxPooling2D)
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [ ]:
simple_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')])

In [ ]:
# Compiling model
simple_model.compile(optimizer = Adam(learning_rate=0.001),
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
simple_model.summary()

In [ ]:
# Preventing overfitting by introducing early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
#training model for 50 epochs
simple_model_history = simple_model.fit(datagen.flow(X_train, y_train, batch_size = 64), epochs=50, validation_data=(X_val, y_val), callbacks=[early_stopping])

In [ ]:
# metric obtained from training
training_accuracy = simple_model_history.history['accuracy']
training_loss = simple_model_history.history['loss']
validation_accuracy = simple_model_history.history['val_accuracy']
validation_loss = simple_model_history.history['val_loss']
print("training accuracy is : ", training_accuracy)
print("training loss is : ", training_loss)
print("validation accuracy is : ", validation_accuracy)
print("validation loss is : ", validation_loss)

In [ ]:
# Model evaluation
test_loss, test_accuracy = simple_model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss:.2f}")
print(f"Test Accuracy: {test_accuracy:.2f}")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score

In [ ]:
y_pred = simple_model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(y_val, y_pred_classes))

In [ ]:
simple_model.save('simple_model.h5')

# Building more complex CNN model

In [ ]:
complex_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

In [ ]:
complex_model.compile(optimizer = Adam(learning_rate=0.001),
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
complex_model.summary()

In [ ]:
# Preventing overfitting by introducing early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
complex_model_history = complex_model.fit(datagen.flow(X_train, y_train, batch_size = 64), epochs = 50, validation_data=(X_val, y_val), callbacks=[early_stopping])

In [ ]:
# Model evaluation
test_loss, test_accuracy = complex_model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss:.2f}")
print(f"Test Accuracy: {test_accuracy:.2f}")

# Finetuning Resnet with Pytorch

In [ ]:
import torch
import copy
import torch.nn as nn
import time
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision.transforms import RandomHorizontalFlip, RandomRotation, ColorJitter
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
# defining function for training and determining the best values
def train_model(model, lossFunction, optimizer, X_train, X_val, y_train, y_val, device, num_epochs=50, patience=2, batch_size=32):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = float('inf')  
    consecutive_epochs_without_improvement = 0

    # Converting data to tensor
    X_train = torch.tensor(X_train, dtype = torch.float32).to(device)
    y_train = torch.tensor(y_train, dtype = torch.long).to(device)
    X_val = torch.tensor(X_val, dtype = torch.float32).to(device)
    y_val = torch.tensor(y_val, dtype = torch.long).to(device)

    # Creating dataloaders
    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)
    train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
    val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False)

    dataloaders = {'train': train_loader, 'val': val_loader}
    dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # training mode
            else:
                model.eval()   # evaluate mode

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = lossFunction(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'val':
                if epoch_loss < best_loss:
                    best_loss = epoch_loss
                    best_model_wts = copy.deepcopy(model.state_dict())
                    consecutive_epochs_without_improvement = 0
                else:
                    consecutive_epochs_without_improvement += 1

        if consecutive_epochs_without_improvement >= patience:
            print(f"Early stopping after {epoch} epochs")
            break

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Loss: {:.4f}'.format(best_loss))

    model.load_state_dict(best_model_wts)

    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in dataloaders['val']:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # classification report and confusion matrix
    
    print(classification_report(all_labels, all_preds))
    
    return model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# Finetuning Resnet101 and adding additional layer

In [ ]:
Resnet101 = models.resnet101(weights=True)
Resnet101

In [ ]:
# finetuning weighting by setting param.requires_grad = True 
for param in Resnet101.parameters():
    param.requires_grad = True

In [ ]:
#Get input features and, replacing last fully connected layer based on model
in_features = Resnet101.fc.in_features
Resnet101.fc = nn.Linear(in_features, len(classes))
# adding additional layer
Resnet101.conv1 = nn.Conv2d(1, 64, kernel_size = 7, stride = 2, padding = 3, bias = False)

In [ ]:
Resnet_fineTuning = Resnet101.to(device)

In [ ]:
#optimzation, learning rate, loss to be used
optimizer = optim.Adam(Resnet_fineTuning.parameters(), lr = 0.00005)
learning_rate = lr_scheduler.StepLR(optimizer, step_size = 7, gamma = 0.1)
Loss_Function = nn.CrossEntropyLoss()

In [ ]:
import torch

In [ ]:
# Convert our numpy arrays to PyTorch tensors
X_train_tensor = torch.from_numpy(X_train).float()
X_val_tensor = torch.from_numpy(X_val).float()

In [ ]:
# Reshape the tensors to [batch_size, channels, height, width]
X_train_tensor = X_train_tensor.permute(0, 3, 1, 2)
X_val_tensor = X_val_tensor.permute(0, 3, 1, 2)

In [ ]:
tuned_model = train_model(Resnet_fineTuning, Loss_Function, optimizer,  X_train_tensor, X_val_tensor, y_train, y_val, device, num_epochs = 1, patience = 3)

In [ ]:
torch.save(tuned_model, "Resnet_fineTuning.pth")

In [ ]:
tuned_model.eval() 

In [ ]:
X_test_tensor = torch.from_numpy(X_test).float()

In [ ]:
# Reshape the tensors to [batch_size, channels, height, width]
X_test_tensor = X_test_tensor.permute(0, 3, 1, 2)

In [ ]:
X_test = torch.tensor(X_test_tensor, dtype = torch.float32).to(device)
y_test = torch.tensor(y_test, dtype = torch.long).to(device)

# Creating test dataloaders
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = True)

test_dataloader = {'test': test_loader}

In [ ]:
# predicting data
from torch.utils.data import DataLoader
y_true = []
y_pred = []
with torch.no_grad():
    for inputs, labels in test_dataloader['test']:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = tuned_model(inputs)
        _, preds = torch.max(outputs, 1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

In [ ]:
classification_report(y_true, y_pred, target_names = classes, output_dict = True)